<a href="https://colab.research.google.com/github/thegregbeyond/FreeFuse-AI-Calbright-Project/blob/main/Object_Detection_V4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 1) Extract Video Stills

In [ ]:
# EXTRACT INDIVIDUAL STILLS FROM EACH VIDEO IN A SOURCE FOLDER.
# CAN DEFINE STARTING SECOND AND INTERVAL BETWEEN STILLS IN SECONDS.

from pathlib import Path
import cv2
from google.colab import drive

# === Configuration ===
INPUT_DIR = Path('/content/drive/MyDrive/FreeFuse_Project/Source_Videos/Alloy Personal Training')
OUTPUT_DIR = Path('/content/drive/MyDrive/FreeFuse_Project/Extracted_Stills')
CAPTURE_INTERVAL_S = 4      # seconds between captures
START_TIME_S = 1            # skip first N seconds of each video
VIDEO_EXTS = {'.mp4', '.mov', '.avi'}


def mount_drive(mount_point: Path = Path('/content/drive')) -> None:
    """
    Mount Google Drive at the given mount_point if not already mounted.
    """
    if not mount_point.exists() or not any(mount_point.iterdir()):
        print("Mounting Google Drive…")
        drive.mount(str(mount_point), force_remount=True)
        print("Drive mounted.")
    else:
        print("Google Drive already mounted.")


def extract_stills(
    input_dir: Path,
    output_dir: Path,
    interval_s: float,
    start_s: float
) -> None:
    """
    Extract frames every `interval_s` seconds from each video in `input_dir`,
    skipping the first `start_s` seconds, and save them to `output_dir` with
    filenames as <video_stem>_<total_seconds_padded_4d>.jpg.
    """
    output_dir.mkdir(parents=True, exist_ok=True)
    video_files = [f for f in sorted(input_dir.iterdir()) if f.suffix.lower() in VIDEO_EXTS]

    if not video_files:
        print(f"No videos found in {input_dir}")
        return

    print(f"Found {len(video_files)} videos in {input_dir}")
    for idx, video_file in enumerate(video_files, start=1):
        print(f"[{idx}/{len(video_files)}] Processing '{video_file.name}'")
        cap = cv2.VideoCapture(str(video_file))
        if not cap.isOpened():
            print(f"  ✗ Could not open {video_file.name}, skipping.")
            continue

        fps = cap.get(cv2.CAP_PROP_FPS) or 30.0
        start_frame = int(start_s * fps)
        if start_frame > 0:
            cap.set(cv2.CAP_PROP_POS_FRAMES, start_frame)
            print(f"  → Skipped first {start_s}s ({start_frame} frames)")

        next_capture = start_s
        saved_count = 0

        while True:
            ret, frame = cap.read()
            if not ret:
                break

            current_s = cap.get(cv2.CAP_PROP_POS_MSEC) / 1000.0
            if current_s >= next_capture:
                total_sec = int(current_s)
                timestamp = f"{total_sec:04d}"
                filename = f"{video_file.stem}_{timestamp}.jpg"
                filepath = output_dir / filename
                cv2.imwrite(str(filepath), frame)
                print(f"  ✔ Saved {filename}")
                saved_count += 1
                next_capture += interval_s

        cap.release()
        print(f"  Completed '{video_file.name}', saved {saved_count} frames.\n")

    print("All videos processed.")


def main() -> None:
    mount_drive()
    extract_stills(INPUT_DIR, OUTPUT_DIR, CAPTURE_INTERVAL_S, START_TIME_S)


if __name__ == '__main__':
    main()

## 2) Object Detection

**Annotation File Schema:**
* frame_id (string): Unique frame identifier (e.g., 'video123_0030').
* track_id (string): Ppersistent ID for unique object instance across multiple frames (for future use)
* object_id (string): Unique identifier for object within this specific frame (e.g., 'video123_0030_obj1').
* timestamp_sec (integer): Time offset in seconds when still was captured.
* image_width_px (integer): Width of still image in pixels.
* image_height_px (integer): Height of still image in pixels.
* machine_id (string): Machine-generated ID for detected object class from knowledge graph (e.g., '/m/0271t').
* object_name (string): Detected object label in its canonical form (e.g., 'coffee_mug').
* object_category (string): Gigh-level grouping for object (e.g., 'electronics', 'furniture').
* x_min (integer): Left coordinate of the bounding box in pixels.
* y_min (integer): Top coordinate of the bounding box in pixels.
* x_max (integer): Right coordinate of the bounding box in pixels.
* y_max (integer): Bottom coordinate of the bounding box in pixels.
* segmentation_mask (string): Pixel-wise mask for object, stored as array of polygon coordinates (e.g., '[[[x1,y1],[x2,y2],...]]').
* confidence (float): Model's confidence score for initial detection (0.0–1.0).
* interaction_score (float): Score (real or mocked) representing object's potential for user engagement (0.0–1.0).

In [ ]:
# CREATES EXPANDED ANNOTATION FILE BASED ON FASTER R-CNN MODEL.
# BASED ON THE OPEN IMAGES V4 MODEL WHICH INCLUDES 600 OBJECT CLASSES.
# OBJECT CATEGORY IS POPULATED BASED ON CANONICAL MAPPING OF THOSE CLASSES.
# NOTE THAT THIS IS A HEAVY-WEIGHT ALGORITHM AND MAY RUN SLOW.

import numpy as np
import pandas as pd
import tensorflow as tf
import tensorflow_hub as hub
from PIL import Image
from pathlib import Path
from google.colab import drive
import json
import random

# ─── Configuration ───────────────────────────────────────────────────────────────
STILLS_DIR      = Path('/content/drive/MyDrive/FreeFuse_Project/Extracted_Stills')
OUTPUT_CSV      = STILLS_DIR / 'final_annotations.csv'
MODEL_URL       = "https://tfhub.dev/google/faster_rcnn/openimages_v4/inception_resnet_v2/1"
MAX_OBJECTS     = 10
CONF_THRESHOLD  = 0.5

# ─── Taxonomy Mapping ───────────────────────────────────────────────────────────
# Maps lower-cased object names to high-level categories
TAXONOMY_MAP = {
    # --- Human & Person ---
    'person': 'human_and_person', 'man': 'human_and_person', 'woman': 'human_and_person',
    'boy': 'human_and_person', 'girl': 'human_and_person',
    'human face': 'human_body_parts', 'human hand': 'human_body_parts', 'human head': 'human_body_parts',
    'human arm': 'human_body_parts', 'human leg': 'human_body_parts', 'human foot': 'human_body_parts',
    'human eye': 'human_body_parts', 'human ear': 'human_body_parts', 'human nose': 'human_body_parts',
    'human mouth': 'human_body_parts', 'human hair': 'human_body_parts', 'human beard': 'human_body_parts',

    # --- Clothing & Accessories ---
    'clothing': 'clothing_and_accessories', 'shirt': 'clothing_and_accessories', 't-shirt': 'clothing_and_accessories',
    'jeans': 'clothing_and_accessories', 'pants': 'clothing_and_accessories', 'dress': 'clothing_and_accessories',
    'skirt': 'clothing_and_accessories', 'jacket': 'clothing_and_accessories', 'coat': 'clothing_and_accessories',
    'footwear': 'clothing_and_accessories', 'boot': 'clothing_and_accessories', 'sandal': 'clothing_and_accessories',
    'high-heeled shoe': 'clothing_and_accessories', 'handbag': 'clothing_and_accessories', 'suitcase': 'clothing_and_accessories',
    'backpack': 'clothing_and_accessories', 'tie': 'clothing_and_accessories', 'belt': 'clothing_and_accessories',
    'hat': 'clothing_and_accessories', 'sunglasses': 'clothing_and_accessories', 'scarf': 'clothing_and_accessories',
    'glove': 'clothing_and_accessories', 'swimwear': 'clothing_and_accessories', 'watch': 'clothing_and_accessories',
    'fedora': 'clothing_and_accessories', 'sombrero': 'clothing_and_accessories', 'trousers': 'clothing_and_accessories',
    'briefcase': 'clothing_and_accessories', 'earrings': 'clothing_and_accessories', 'necklace': 'clothing_and_accessories',
    'ring': 'clothing_and_accessories', 'wallet': 'clothing_and_accessories', 'suit': 'clothing_and_accessories',
    'glasses': 'clothing_and_accessories', 'fashion accessory': 'clothing_and_accessories', 'shorts': 'clothing_and_accessories',
    'brassiere': 'clothing_and_accessories', 'luggage and bags': 'clothing_and_accessories', 'helmet': 'clothing_and_accessories',
    'high heels': 'clothing_and_accessories',

    # --- Animals ---
    'animal': 'animals', 'cat': 'animals', 'dog': 'animals', 'bird': 'animals', 'duck': 'animals',
    'horse': 'animals', 'sheep': 'animals', 'cow': 'animals', 'elephant': 'animals', 'bear': 'animals',
    'zebra': 'animals', 'giraffe': 'animals', 'monkey': 'animals', 'pig': 'animals', 'lion': 'animals',
    'tiger': 'animals', 'penguin': 'animals', 'fish': 'animals', 'shark': 'animals', 'whale': 'animals',
    'insect': 'animals', 'butterfly': 'animals', 'squirrel': 'animals', 'rabbit': 'animals', 'ant': 'animals',
    'lizard': 'animals', 'snake': 'animals', 'spider': 'animals', 'tortoise': 'animals', 'bee': 'animals',
    'camel': 'animals', 'crocodile': 'animals', 'dolphin': 'animals', 'eagle': 'animals', 'fox': 'animals',
    'frog': 'animals', 'hamster': 'animals', 'koala': 'animals', 'leopard': 'animals', 'mouse': 'animals',
    'otter': 'animals', 'owl': 'animals', 'parrot': 'animals', 'raccoon': 'animals', 'seal': 'animals',
    'starfish': 'animals', 'swan': 'animals', 'turtle': 'animals', 'wolf': 'animals', 'goldfish': 'animals',
    'jellyfish': 'animals', 'sea lion': 'animals', 'sea turtle': 'animals', 'seahorse': 'animals',
    'shrimp': 'animals', 'turkey': 'animals', 'woodpecker': 'animals', 'cattle': 'animals', 'sparrow': 'animals',

    # --- Vehicles ---
    'vehicle': 'vehicles', 'car': 'vehicles', 'truck': 'vehicles', 'bus': 'vehicles', 'van': 'vehicles',
    'motorcycle': 'vehicles', 'bicycle': 'vehicles', 'train': 'vehicles', 'airplane': 'vehicles', 'boat': 'vehicles',
    'helicopter': 'vehicles', 'taxi': 'vehicles', 'ambulance': 'vehicles', 'tractor': 'vehicles', 'forklift': 'vehicles',
    'limousine': 'vehicles', 'school bus': 'vehicles', 'snowmobile': 'vehicles', 'tank': 'vehicles', 'unicycle': 'vehicles',
    'yacht': 'vehicles', 'wheel': 'vehicles', 'bicycle wheel': 'vehicles', 'tire': 'vehicles', 'auto part': 'vehicles',
    'land vehicle': 'vehicles',

    # --- Electronics & Tech ---
    'electronics': 'electronics_and_tech', 'computer': 'electronics_and_tech', 'laptop': 'electronics_and_tech',
    'computer keyboard': 'electronics_and_tech', 'computer mouse': 'electronics_and_tech', 'tablet computer': 'electronics_and_tech',
    'mobile phone': 'electronics_and_tech', 'telephone': 'electronics_and_tech', 'television': 'electronics_and_tech',
    'camera': 'electronics_and_tech', 'projector': 'electronics_and_tech', 'remote control': 'electronics_and_tech',
    'headphones': 'electronics_and_tech', 'speaker': 'electronics_and_tech', 'microphone': 'electronics_and_tech',
    'webcam': 'electronics_and_tech', 'computer monitor': 'electronics_and_tech', 'cassette deck': 'electronics_and_tech',

    # --- Furniture ---
    'furniture': 'furniture', 'chair': 'furniture', 'table': 'furniture', 'couch': 'furniture', 'desk': 'furniture',
    'bed': 'furniture', 'bookcase': 'furniture', 'drawer': 'furniture', 'cabinetry': 'furniture', 'stool': 'furniture',
    'bench': 'furniture', 'shelf': 'furniture', 'filing cabinet': 'furniture', 'armchair': 'furniture',
    'rocking chair': 'furniture', 'sofa bed': 'furniture',

    # --- Food & Drink ---
    'food': 'food_and_drink', 'fruit': 'food_and_drink', 'apple': 'food_and_drink', 'banana': 'food_and_drink',
    'orange': 'food_and_drink', 'grape': 'food_and_drink', 'strawberry': 'food_and_drink', 'vegetable': 'food_and_drink',
    'carrot': 'food_and_drink', 'tomato': 'food_and_drink', 'potato': 'food_and_drink', 'bell pepper': 'food_and_drink',
    'bread': 'food_and_drink', 'pizza': 'food_and_drink', 'pasta': 'food_and_drink', 'sandwich': 'food_and_drink',
    'cake': 'food_and_drink', 'cookie': 'food_and_drink', 'doughnut': 'food_and_drink', 'coffee': 'food_and_drink',
    'tea': 'food_and_drink', 'juice': 'food_and_drink', 'wine': 'food_and_drink', 'beer': 'food_and_drink', 'bottle': 'food_and_drink',
    'sushi': 'food_and_drink', 'cheese': 'food_and_drink', 'hot dog': 'food_and_drink', 'ice cream': 'food_and_drink',
    'pancake': 'food_and_drink', 'popcorn': 'food_and_drink', 'pretzel': 'food_and_drink', 'salad': 'food_and_drink', 'waffle': 'food_and_drink',
    'bacon': 'food_and_drink', 'burrito': 'food_and_drink', 'taco': 'food_and_drink', 'hamburger': 'food_and_drink',
    'candy': 'food_and_drink', 'guacamole': 'food_and_drink', 'fast food': 'food_and_drink', 'drink': 'food_and_drink',
    'cocktail': 'food_and_drink',

    # --- Kitchen & Tableware ---
    'kitchenware': 'kitchen_and_tableware', 'plate': 'kitchen_and_tableware', 'bowl': 'kitchen_and_tableware',
    'cup': 'kitchen_and_tableware', 'mug': 'kitchen_and_tableware', 'fork': 'kitchen_and_tableware',
    'spoon': 'kitchen_and_tableware', 'knife': 'kitchen_and_tableware', 'cutting board': 'kitchen_and_tableware',
    'teapot': 'kitchen_and_tableware', 'toaster': 'kitchen_and_tableware', 'microwave oven': 'kitchen_and_tableware',
    'refrigerator': 'kitchen_and_tableware', 'oven': 'kitchen_and_tableware', 'blender': 'kitchen_and_tableware',
    'sink': 'kitchen_and_tableware', 'wine glass': 'kitchen_and_tableware', 'frying pan': 'kitchen_and_tableware', 'ladle': 'kitchen_and_tableware',
    'coffee cup': 'kitchen_and_tableware', 'tableware': 'kitchen_and_tableware', 'saucer': 'kitchen_and_tableware',

    # --- Tools & Equipment ---
    'tool': 'tools_and_equipment', 'hammer': 'tools_and_equipment', 'screwdriver': 'tools_and_equipment',
    'wrench': 'tools_and_equipment', 'saw': 'tools_and_equipment', 'power tool': 'tools_and_equipment',
    'drill': 'tools_and_equipment', 'ladder': 'tools_and_equipment', 'axe': 'tools_and_equipment', 'chisel': 'tools_and_equipment',

    # --- Sports & Recreation ---
    'sports equipment': 'sports_and_recreation', 'ball': 'sports_and_recreation', 'football': 'sports_and_recreation',
    'baseball': 'sports_and_recreation', 'basketball': 'sports_and_recreation', 'tennis ball': 'sports_and_recreation',
    'surfboard': 'sports_and_recreation', 'skateboard': 'sports_and_recreation', 'bicycle helmet': 'sports_and_recreation',
    'ski': 'sports_and_recreation', 'snowboard': 'sports_and_recreation', 'dumbbell': 'sports_and_recreation',
    'kite': 'sports_and_recreation', 'baseball bat': 'sports_and_recreation', 'baseball glove': 'sports_and_recreation',
    'tennis racket': 'sports_and_recreation', 'billiard table': 'sports_and_recreation', 'bowling ball': 'sports_and_recreation',
    'cricket ball': 'sports_and_recreation', 'frisbee': 'sports_and_recreation', 'golf ball': 'sports_and_recreation',
    'rugby ball': 'sports_and_recreation', 'volleyball (ball)': 'sports_and_recreation', 'roller skates': 'sports_and_recreation',
    'hiking equipment': 'sports_and_recreation', 'treadmill': 'sports_and_recreation',

    # --- Musical Instruments ---
    'musical instrument': 'musical_instruments', 'guitar': 'musical_instruments', 'piano': 'musical_instruments',
    'violin': 'musical_instruments', 'drum': 'musical_instruments', 'saxophone': 'musical_instruments',
    'trumpet': 'musical_instruments', 'flute': 'musical_instruments', 'accordion': 'musical_instruments',
    'cello': 'musical_instruments', 'harp': 'musical_instruments', 'trombone': 'musical_instruments', 'xylophone': 'musical_instruments',

    # --- Buildings & Structures ---
    'building': 'buildings_and_structures', 'house': 'buildings_and_structures', 'skyscraper': 'buildings_and_structures',
    'tower': 'buildings_and_structures', 'bridge': 'buildings_and_structures', 'castle': 'buildings_and_structures',
    'church': 'buildings_and_structures', 'stadium': 'buildings_and_structures', 'stairs': 'buildings_and_structures',
    'door': 'buildings_and_structures', 'window': 'buildings_and_structures', 'wall': 'buildings_and_structures',
    'lighthouse': 'buildings_and_structures', 'mosque': 'buildings_and_structures', 'temple': 'buildings_and_structures',
    'billboard': 'buildings_and_structures', 'office building': 'buildings_and_structures', 'convenience store': 'buildings_and_structures',

    # --- Home & Office Goods ---
    'office supplies': 'home_and_office_goods', 'pen': 'home_and_office_goods', 'paper': 'home_and_office_goods',
    'book': 'home_and_office_goods', 'vase': 'home_and_office_goods', 'clock': 'home_and_office_goods',
    'picture frame': 'home_and_office_goods', 'lamp': 'home_and_office_goods', 'potted plant': 'home_and_office_goods',
    'mirror': 'home_and_office_goods', 'towel': 'home_and_office_goods', 'scissors': 'home_and_office_goods',
    'candle': 'home_and_office_goods', 'pillow': 'home_and_office_goods', 'rug': 'home_and_office_goods',
    'toilet paper': 'home_and_office_goods', 'umbrella': 'home_and_office_goods', 'houseplant': 'home_and_office_goods',
    'flowerpot': 'home_and_office_goods', 'mechanical fan': 'home_and_office_goods', 'box': 'home_and_office_goods',
    'poster': 'home_and_office_goods', 'wall clock': 'home_and_office_goods', 'waste container': 'home_and_office_goods',
    'flag': 'home_and_office_goods', 'barrel': 'home_and_office_goods',

    # --- Outdoor & Nature ---
    'tree': 'outdoor_and_nature', 'flower': 'outdoor_and_nature', 'plant': 'outdoor_and_nature', 'mountain': 'outdoor_and_nature',
    'volcano': 'outdoor_and_nature', 'beach': 'outdoor_and_nature', 'river': 'outdoor_and_nature', 'lake': 'outdoor_and_nature',
    'sky': 'outdoor_and_nature', 'traffic light': 'outdoor_and_nature', 'fire hydrant': 'outdoor_and_nature', 'fountain': 'outdoor_and_nature',
    'palm tree': 'outdoor_and_nature', 'rose': 'outdoor_and_nature', 'sunflower': 'outdoor_and_nature', 'maple': 'outdoor_and_nature',
    'common fig': 'outdoor_and_nature',

    # --- Toys & Games ---
    'toy': 'toys_and_games', 'doll': 'toys_and_games', 'teddy bear': 'toys_and_games', 'lego': 'toys_and_games',
    'dice': 'toys_and_games', 'chess': 'toys_and_games', 'puzzle': 'toys_and_games', 'swing': 'toys_and_games',
    'balloon': 'toys_and_games',

    # --- Appliances ---
    'home appliance': 'appliances',

    # --- Medical ---
    'wheelchair': 'medical',

    # --- Military & Weaponry ---
    'rocket': 'military_and_weaponry', 'missile': 'military_and_weaponry'
}
DEFAULT_CATEGORY = 'other'
# ────────────────────────────────────────────────────────────────────────────────

def mount_drive():
    """Mount Google Drive in Colab if needed."""
    if not Path('/content/drive').is_mount():
        print("Mounting Google Drive…")
        drive.mount('/content/drive', force_remount=True)
    print("Google Drive mounted.\n")


def load_model(url: str):
    """Load TF-Hub detection model and return its default signature."""
    print(f"Loading model from TF-Hub: {url}")
    model = hub.load(url)
    print("Model loaded.\n")
    return model.signatures['default']


def parse_frame_and_timestamp(fname: str):
    """
    Given 'video123_0030.jpg', returns ('video123_0030', 30).
    Falls back to (stem, 0).
    """
    stem = Path(fname).stem
    parts = stem.rsplit('_', 1)
    if len(parts) == 2 and parts[1].isdigit():
        mmss = parts[1]
        secs = int(mmss[-2:]) + (int(mmss[:-2]) * 60)
        return stem, secs
    return stem, 0


def process_image(path: Path, detector) -> list[dict]:
    """
    Run detection on one image and return annotations matching schema.
    """
    img = np.array(Image.open(path).convert('RGB'), dtype=np.float32) / 255.0
    h, w, _ = img.shape
    frame_id, ts = parse_frame_and_timestamp(path.name)

    inputs = tf.expand_dims(tf.convert_to_tensor(img), 0)
    results = detector(images=inputs)

    scores   = results['detection_scores'].numpy().reshape(-1)
    boxes    = results['detection_boxes'].numpy().reshape(-1, 4)
    names    = results['detection_class_names'].numpy().reshape(-1)
    entities = results['detection_class_entities'].numpy().reshape(-1)

    labels = [nm.decode() if isinstance(nm, (bytes, bytearray)) else str(nm) for nm in names]
    mids   = [ent.decode() if isinstance(ent, (bytes, bytearray)) else str(ent) for ent in entities]

    annots, kept = [], 0
    for i, score in enumerate(scores):
        if kept >= MAX_OBJECTS or score < CONF_THRESHOLD:
            continue
        ymin, xmin, ymax, xmax = boxes[i]
        x_min = int(xmin * w)
        y_min = int(ymin * h)
        x_max = int(xmax * w)
        y_max = int(ymax * h)

        segmentation = json.dumps([[[x_min, y_min], [x_max, y_min], [x_max, y_max], [x_min, y_max]]])
        interaction = float(round(random.random(), 4))

        # snake-case object_name
        raw_name = mids[i]
        snake_name = raw_name.lower().replace(' ', '_').replace('-', '_')

        annots.append({
            'frame_id':          frame_id,
            'track_id':          "",
            'object_id':         f"{frame_id}_obj{kept+1}",
            'timestamp_sec':     ts,
            'image_width_px':    w,
            'image_height_px':   h,
            'machine_id':        labels[i].lower().replace(' ', '_'),
            'object_name':       snake_name,
            'object_category':   'N/A',  # placeholder
            'x_min':             x_min,
            'y_min':             y_min,
            'x_max':             x_max,
            'y_max':             y_max,
            'segmentation_mask': segmentation,
            'confidence':        float(round(score, 4)),
            'interaction_score': interaction,
        })
        kept += 1

    return annots


def main():
    mount_drive()
    detector = load_model(MODEL_URL)

    images = sorted(STILLS_DIR.glob('*.jpg'))
    if not images:
        print(f"No images found in {STILLS_DIR}")
        return

    print(f"Processing {len(images)} images…")
    all_annots = []
    for idx, img_path in enumerate(images, start=1):
        print(f" [{idx}/{len(images)}] {img_path.name}")
        all_annots.extend(process_image(img_path, detector))

    if not all_annots:
        print("No detections passed the threshold.")
        return

    df = pd.DataFrame(all_annots)

    # fill missing confidences
    if df['confidence'].isnull().any():
        df['confidence'].fillna(df['confidence'].median(), inplace=True)

    # enforce column order
    schema_cols = [
        'frame_id', 'track_id', 'object_id',
        'timestamp_sec',
        'image_width_px', 'image_height_px',
        'machine_id',
        'object_name', 'object_category',
        'x_min', 'y_min', 'x_max', 'y_max',
        'segmentation_mask',
        'confidence', 'interaction_score'
    ]
    df = df[schema_cols]

    # ensure object_name is snake_case (just in case)
    df['object_name'] = (
        df['object_name']
          .str.lower()
          .str.replace(r'[^a-z0-9]+', '_', regex=True)
          .str.strip('_')
    )

    # map categories
    df['object_category'] = (
        df['object_name']
          .str.replace('_', ' ')
          .map(TAXONOMY_MAP)
          .fillna(DEFAULT_CATEGORY)
    )

    print(f"\nSaving {len(df)} annotations to {OUTPUT_CSV}")
    df.to_csv(OUTPUT_CSV, index=False)
    print("Done.")

if __name__ == '__main__':
    main()

Google Drive mounted.

Loading model from TF-Hub: https://tfhub.dev/google/faster_rcnn/openimages_v4/inception_resnet_v2/1
Model loaded.

Processing 1 images…
 [1/1] 852424-hd_1920_1080_24fps_0007.jpg

Saving 8 annotations to /content/drive/MyDrive/FreeFuse_Project/Extracted_Stills/final_annotations.csv
Done.


## 3) Label Stills

In [ ]:
# DRAWS POLYGONS ON EXTRACTED STILLS BASED ON ANNOTATION FILE.
# ADDS LABEL WITH CONFIDENCE SCORE.

import json
from pathlib import Path
import cv2
import pandas as pd
from google.colab import drive

# === Configuration ===
STILLS_DIR       = Path('/content/drive/MyDrive/FreeFuse_Project/Extracted_Stills')
ANNOTATIONS_CSV  = STILLS_DIR / 'draft_annotations.csv'
OUTPUT_DIR       = Path('/content/drive/MyDrive/FreeFuse_Project/Labeled_Stills')

# Drawing settings
BOX_COLOR        = (0, 255, 0)        # BGR green
TEXT_COLOR       = (255, 255, 255)    # BGR white
BOX_THICKNESS    = 2
FONT             = cv2.FONT_HERSHEY_SIMPLEX
FONT_SCALE       = 0.6
LINE_TYPE        = cv2.LINE_AA
TEXT_PADDING     = 4

# === Helpers ===
def mount_drive(mount_point: Path = Path('/content/drive')) -> None:
    """
    Mount Google Drive if not already.
    """
    if not mount_point.exists() or not any(mount_point.iterdir()):
        print("Mounting Google Drive…")
        drive.mount(str(mount_point), force_remount=True)
        print("Drive mounted.")
    else:
        print("Google Drive already mounted.")


def load_annotations(csv_path: Path) -> pd.DataFrame:
    """
    Load annotations CSV and validate schema.
    """
    if not csv_path.exists():
        raise FileNotFoundError(f"Annotations file not found: {csv_path}")
    df = pd.read_csv(csv_path)

    expected = [
        'frame_id','track_id','object_id','timestamp_sec',
        'image_width_px','image_height_px','machine_id',
        'object_name','object_category','x_min','y_min','x_max','y_max',
        'segmentation_mask','confidence','interaction_score'
    ]
    missing = set(expected) - set(df.columns)
    if missing:
        raise ValueError(f"Missing columns: {missing}")
    # derive image filename from frame_id
    df['image_file'] = df['frame_id'].astype(str) + '.jpg'
    return df


def annotate_images(df: pd.DataFrame, stills_dir: Path, output_dir: Path) -> None:
    """
    Draw polygons and labels on each still image and save.
    """
    output_dir.mkdir(parents=True, exist_ok=True)
    grouped = df.groupby('image_file')
    print(f"Found {len(grouped)} images to annotate.")

    for idx, (img_name, group) in enumerate(grouped, start=1):
        img_path = stills_dir / img_name
        print(f"[{idx}/{len(grouped)}] Annotating {img_path.name}")
        img = cv2.imread(str(img_path))
        if img is None:
            print(f"  ✗ Cannot load {img_path}")
            continue

        for _, row in group.iterrows():
            # parse polygon mask
            try:
                polygons = json.loads(row['segmentation_mask'])
            except json.JSONDecodeError:
                continue

            # draw each polygon
            for poly in polygons:
                pts = [(int(x), int(y)) for x, y in poly]
                pts_np = cv2.UMat(np.array(pts, dtype=np.int32).reshape(-1,1,2))
                cv2.polylines(img, [pts_np.get()], isClosed=True, color=BOX_COLOR, thickness=BOX_THICKNESS)

                # label at first point
                label = f"{row['object_name']}:{row['confidence']:.2f}"
                org = pts[0]
                # compute text size
                (tw, th), _ = cv2.getTextSize(label, FONT, FONT_SCALE, thickness=1)
                # background rectangle
                x0, y0 = org
                x1, y1 = x0 + tw + TEXT_PADDING, y0 + th + TEXT_PADDING
                cv2.rectangle(img, (x0, y0), (x1, y1), BOX_COLOR, thickness=-1)
                # put text
                text_org = (x0 + TEXT_PADDING//2, y0 + th)
                cv2.putText(img, label, text_org, FONT, FONT_SCALE, TEXT_COLOR, thickness=1, lineType=LINE_TYPE)

        out_path = output_dir / img_name.replace('.jpg', '_annotated.jpg')
        cv2.imwrite(str(out_path), img)

    print("Annotation complete.")


def main() -> None:
    mount_drive()
    df = load_annotations(ANNOTATIONS_CSV)
    annotate_images(df, STILLS_DIR, OUTPUT_DIR)


if __name__ == '__main__':
    main()

Google Drive already mounted.
Found 1 images to annotate.
[1/1] Annotating 852424-hd_1920_1080_24fps_0007.jpg
Annotation complete.


# 4) Generate Proxy Interaction Scores

In [1]:
import pandas as pd
import numpy as np
from pathlib import Path
from google.colab import drive

# ─── USER CONFIG ────────────────────────────────────────────────────────────────
DRIVE_MOUNT_POINT = "/content/drive"
INPUT_CSV_PATH    = Path(f"{DRIVE_MOUNT_POINT}/MyDrive/FreeFuse_Project/Extracted_Stills/temp_annotations.csv")
OUTPUT_CSV_PATH   = INPUT_CSV_PATH.parent / "final_annotations.csv"
# ────────────────────────────────────────────────────────────────────────────────

# Base scores by category
CATEGORY_BASE = {
    'electronics_and_tech':     0.70,
    'food_and_drink':           0.60,
    'clothing_and_accessories': 0.50,
    'kitchen_and_tableware':    0.50,
    'sports_and_recreation':    0.50,
    'home_and_office_goods':    0.40,
    'vehicles':                 0.30,
    'tools_and_equipment':      0.30,
    'furniture':                0.20,
    'toys_and_games':           0.40,
    'musical_instruments':      0.40,
    'other':                    0.10,
}
DEFAULT_BASE = 0.10

# Modifier maxima
CENTRALITY_BONUS_MAX   = 0.20
PROMINENCE_BONUS_MAX   = 0.15
HALF_DIAGONAL_DISTANCE = np.sqrt(2) / 2  # ≈0.7071

# ─── DRIVE ───────────────────────────────────────────────────────────────────────
def mount_drive():
    if not Path(DRIVE_MOUNT_POINT).is_mount():
        print("Mounting Google Drive…")
        drive.mount(DRIVE_MOUNT_POINT, force_remount=True)
    else:
        print("Google Drive already mounted.")

# ─── FEATURE COMPUTATION ────────────────────────────────────────────────────────
def compute_spatial_features(df: pd.DataFrame) -> pd.DataFrame:
    df = df.copy()
    df['center_x']         = df['x_min'] + (df['x_max'] - df['x_min']) / 2
    df['center_y']         = df['y_min'] + (df['y_max'] - df['y_min']) / 2
    df['norm_center_x']    = df['center_x'] / df['image_width_px']
    df['norm_center_y']    = df['center_y'] / df['image_height_px']
    df['bb_area_px']       = (df['x_max'] - df['x_min']) * (df['y_max'] - df['y_min'])
    df['relative_bb_area'] = df['bb_area_px'] / (df['image_width_px'] * df['image_height_px'])
    return df

# ─── SCORING ────────────────────────────────────────────────────────────────────
def category_base_score(cat: str) -> float:
    return CATEGORY_BASE.get(cat, DEFAULT_BASE)

def centrality_bonus(norm_x: float, norm_y: float) -> float:
    dist  = np.hypot(norm_x - 0.5, norm_y - 0.5)
    bonus = (1 - dist / HALF_DIAGONAL_DISTANCE) * CENTRALITY_BONUS_MAX
    return float(max(0.0, bonus))

def prominence_bonus(rel_area: float) -> float:
    return float(np.sqrt(rel_area) * PROMINENCE_BONUS_MAX)

def compute_interaction_scores(df: pd.DataFrame) -> pd.DataFrame:
    raw = []
    for _, r in df.iterrows():
        # people/human categories get 0
        if 'human' in str(r['object_category']):
            raw.append(0.0)
        else:
            base  = category_base_score(r['object_category'])
            cent  = centrality_bonus(r['norm_center_x'], r['norm_center_y'])
            prom  = prominence_bonus(r['relative_bb_area'])
            raw_score = (base + cent + prom) * r['confidence']
            raw.append(raw_score)
    df['interaction_score_raw'] = raw

    # normalize to [0,1]
    mn, mx = df['interaction_score_raw'].min(), df['interaction_score_raw'].max()
    if mx > mn:
        df['interaction_score'] = ((df['interaction_score_raw'] - mn) / (mx - mn) * 0.99 + 0.01).round(4)
    else:
        df['interaction_score'] = 0.0

    # ensure humans stay at 0
    human_mask = df['object_category'].str.contains('human', na=False)
    df.loc[human_mask, 'interaction_score'] = 0.0

    return df

# ─── TRACK / ID ASSIGNMENT ──────────────────────────────────────────────────────
def assign_ids(df: pd.DataFrame) -> pd.DataFrame:
    """
    - frame_id: already present
    - object_id: already present
    - track_id: mocked as '<frame_id>_<object_name>_<n>' per frame
    - segmentation_mask: empty JSON string
    """
    df = df.copy()
    df['segmentation_mask'] = '[]'
    df['machine_id']        = ''  # if you have a MID column, copy it here
    df['track_id']          = None

    # within each frame and object_name, number them
    def make_track(sub):
        counts = {}
        tracks = []
        for obj, oid in zip(sub['object_name'], sub['object_id']):
            counts[obj] = counts.get(obj, 0) + 1
            tracks.append(f"{sub.name}_{obj}_{counts[obj]}")
        return tracks

    # groupby frame_id
    tracks = []
    for frame, group in df.groupby('frame_id'):
        group = group.copy()
        group.name = frame
        tracks.extend(make_track(group))

    df['track_id'] = tracks
    return df

# ─── MAIN ───────────────────────────────────────────────────────────────────────
def main():
    mount_drive()

    try:
        df = pd.read_csv(INPUT_CSV_PATH)
        print(f"Loaded {len(df)} rows from {INPUT_CSV_PATH.name}")
    except FileNotFoundError:
        print(f"Missing input: {INPUT_CSV_PATH}")
        return

    # compute features + scores + IDs
    df = compute_spatial_features(df)
    df = compute_interaction_scores(df)
    df = assign_ids(df)

    # drop helper columns
    drop_cols = [
        'center_x','center_y',
        'norm_center_x','norm_center_y',
        'bb_area_px','relative_bb_area',
        'interaction_score_raw'
    ]
    df.drop(columns=drop_cols, inplace=True)

    # enforce final schema
    schema = [
        'frame_id','track_id','object_id','timestamp_sec',
        'image_width_px','image_height_px','machine_id',
        'object_name','object_category',
        'x_min','y_min','x_max','y_max',
        'segmentation_mask','confidence','interaction_score'
    ]
    df = df[schema]

    df.to_csv(OUTPUT_CSV_PATH, index=False)
    print(f"Saved {len(df)} scored annotations to {OUTPUT_CSV_PATH}")

if __name__ == "__main__":
    main()

Mounting Google Drive…
Mounted at /content/drive
Loaded 2176 rows from temp_annotations.csv
Saved 2176 scored annotations to /content/drive/MyDrive/FreeFuse_Project/Extracted_Stills/final_annotations.csv


# 5) Generate Week 4 Deliverable

In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from pathlib import Path
from google.colab import drive

# ─── USER CONFIGURATION ─────────────────────────────────────────────────────────
INPUT_CSV_PATH     = Path("/content/drive/MyDrive/FreeFuse_Project/Extracted_Stills/final_annotations.csv")
OUTPUT_MATRIX_PNG  = Path("/content/drive/MyDrive/FreeFuse_Project/insight_prioritization_matrix.png")
# ────────────────────────────────────────────────────────────────────────────────

def mount_drive(mount_point: str = "/content/drive") -> None:
    """Mount Google Drive in Colab if not already mounted."""
    mount_point = Path(mount_point)
    if not mount_point.is_mount():
        print("Mounting Google Drive…")
        drive.mount(str(mount_point), force_remount=True)
    else:
        print("Google Drive already mounted.")

def validate_dataframe(df: pd.DataFrame) -> None:
    """Ensure required columns exist in the DataFrame."""
    required = {
        "object_id", "object_category", "interaction_score"
    }
    missing = required - set(df.columns)
    if missing:
        raise ValueError(f"Missing required columns in input CSV: {missing}")

def prepare_plot_data(df: pd.DataFrame) -> pd.DataFrame:
    """
    Aggregates by object_category to compute:
      - frequency (count of detections)
      - mean_interaction_score
      - log10 of frequency for plotting
    Filters out any 'human' categories.
    """
    # Filter out human-related categories
    mask = ~df["object_category"].str.contains("human", na=False)
    df_filtered = df.loc[mask, :]

    agg = (
        df_filtered
        .groupby("object_category", as_index=False)
        .agg(
            frequency=("object_id", "nunique"),
            mean_interaction_score=("interaction_score", "mean")
        )
    )
    agg["log_frequency"] = np.log10(agg["frequency"].replace(0, np.nan)).fillna(0)
    return agg

def generate_matrix_plot(plot_df: pd.DataFrame, output_path: Path) -> None:
    """
    Creates and saves a 2×2 prioritization matrix:
      - x-axis: log10(frequency)
      - y-axis: mean interaction score
      - bubble size: frequency
      - quadrants defined by median splits
    """
    if plot_df.empty:
        print("No data to plot.")
        return

    # Set dark background and figure size
    plt.style.use("dark_background")
    fig, ax = plt.subplots(figsize=(14, 10))

    # Scatter
    sns.scatterplot(
        data=plot_df,
        x="log_frequency",
        y="mean_interaction_score",
        size="frequency",
        sizes=(100, 1500),
        hue="object_category",
        alpha=0.8,
        palette="tab10",
        ax=ax,
        legend="brief"
    )

    # Annotations
    for _, row in plot_df.iterrows():
        ax.text(
            row.log_frequency,
            row.mean_interaction_score + 0.02,
            row.object_category,
            ha="center",
            va="bottom",
            fontsize=9,
            color="white"
        )

    # Quadrant lines
    x_med = plot_df.log_frequency.median()
    y_med = plot_df.mean_interaction_score.median()
    ax.axvline(x_med, color="red", ls="--", lw=1)
    ax.axhline(y_med, color="red", ls="--", lw=1)

    # Quadrant labels
    x0, x1 = ax.get_xlim()
    y0, y1 = ax.get_ylim()
    ax.text(x1, y1, "Prioritize", ha="right", va="top", color="lightgreen", weight="bold")
    ax.text(x0, y1, "Explore",    ha="left",  va="top", color="skyblue",   weight="bold")
    ax.text(x1, y0, "Automate",   ha="right", va="bottom", color="orange",   weight="bold")
    ax.text(x0, y0, "Deprioritize", ha="left", va="bottom", color="gray",     weight="bold")

    # Labels & legend
    ax.set_title("Object Prioritization Matrix", fontsize=18, pad=16)
    ax.set_xlabel("Log₁₀(Object Frequency)", fontsize=14)
    ax.set_ylabel("Mean Interaction Score",   fontsize=14)
    ax.legend(title="Category", bbox_to_anchor=(1.05, 1), loc="upper left")
    plt.tight_layout()

    # Save
    fig.savefig(str(output_path), dpi=300)
    plt.close(fig)
    print(f"Matrix plot saved to {output_path}")

def main():
    mount_drive()
    try:
        df = pd.read_csv(INPUT_CSV_PATH)
    except FileNotFoundError:
        print(f"Input CSV not found at {INPUT_CSV_PATH}")
        return

    try:
        validate_dataframe(df)
    except ValueError as err:
        print(f"Validation error: {err}")
        return

    plot_df = prepare_plot_data(df)
    print(f"Prepared data for {len(plot_df)} categories.")
    generate_matrix_plot(plot_df, OUTPUT_MATRIX_PNG)

if __name__ == "__main__":
    main()

Google Drive already mounted.
Prepared data for 16 categories.
Matrix plot saved to /content/drive/MyDrive/FreeFuse_Project/insight_prioritization_matrix.png
